<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/GenAI_Source_code_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install openai tiktoken chromadb langchain flask GitPython python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.parsers import LanguageParser
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory

In [4]:
!mkdir test_repo

In [5]:
Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path='test_repo')

<git.repo.base.Repo '/content/test_repo/.git'>

In [6]:
repo_path="/content/test_repo"
loader=GenericLoader.from_filesystem(
    repo_path+'/src/mlProject',
    glob='**/*',
    suffixes=['.py'],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [8]:
document=loader.load()

In [10]:
#perform context aware chunking

In [12]:
document_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=3000,
                                                               chunk_overlap=50)
texts=document_splitter.split_documents(document)

In [15]:
from google.colab import userdata
openai=userdata.get('openai')
os.environ['OPENAI_API_KEY']=openai

In [16]:
embedding=OpenAIEmbeddings(disallowed_special=())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [18]:
## Knowledge base (vector DB)
vectordb=Chroma.from_documents(texts, embedding=embedding, persist_directory='./data')
vectordb.persist()

In [35]:
#llm wrapper
#llm=ChatOpenAI(model_name='gpt-4')
llm=ChatOpenAI()
memory=ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

In [39]:
from langchain.chains import ConversationalRetrievalChain
qa=ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k":3}), memory=memory)

In [42]:
#Q and A
question="How can i code the data ingestion file?"


In [43]:
result=qa(question)
print(result['answer'])

To code the `DataIngestion` class, you can follow the example below. This class will have methods to download a file and extract a zip file based on the configuration provided:

```python
from mlProject.utils.common import download_file, extract_zip_file

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        # Implement the download logic using the source URL from the config
        download_file(self.config.source_URL, self.config.local_data_file)

    def extract_zip_file(self):
        # Implement the extraction logic using the unzip directory from the config
        extract_zip_file(self.config.local_data_file, self.config.unzip_dir)
```

Make sure you have the necessary methods like `download_file` and `extract_zip_file` implemented in your project's utilities or components to handle the actual downloading and extraction of files.
